# This code takes the weekly sea ice concentrations of the ASI passive microwave sea ice concentration products and RSAT-2 CNN ice/water classification and compares them to the CIS weekly regional charts for the area of Coronation Gulf between 2009 and 2020.

In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from datetime import datetime
import numpy as np

Read the dataframes

In [ ]:
cis = pd.read_pickle(r'../../data/WeeklySIC_CIS.pkl')
asi = pd.read_pickle(r'../../data/WeeklySIC_ASI.pkl')
r2 = pd.read_pickle(r'../../data/WeeklySIC_RSAT-2.pkl')

## Figure: Time series of the sea ice concentration of Coronation Gulf for the CIS regional charts, the RSAT-2 CNN classification and ASI SIC products.

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

ax.fill_between(cis['Date'],cis['SIC'])
ax.plot(asi['Date (YYYY-MM-DD)'],asi['Mean Ice Concentration (%)'],color='orange', linewidth=2)
ax.plot(pd.to_datetime(r2['Date (YYYY-MM-DD)']),r2['Mean Ice Concentration (%)'],'+k', markeredgewidth=2)
ax.set_ylim(0,120)
ax.set_xlim(datetime(2008,6,1), datetime(2020,8,31))
ax.set_xticklabels(np.arange(2008,2022,2).tolist(),fontsize=24,fontweight='bold')
ax.set_yticklabels(np.arange(0,120,20),fontsize=24,fontweight='bold')
ax.legend(['ASI', 'RSAT-2', 'CIS'], fontsize=22, loc='upper center', ncol=3)
ax.set_ylabel('SIC (%)', fontsize=24, fontweight='bold')

<center><img src="../../Data/figures/Figure8.png" height="500px"></center>

<center>Figure 8. Timeseries of weekly SIC for all of Coronation Gulf from the CIS regional charts, ASI passive microwave product and output of the RSAT-2 images using the 4-band input CNN model. The values are valid on the Monday for each week. The ASI and RSAT-2 products consist of weekly pixel average.</center>

## Results: Mean bias and RMSE between the RSAT-2/ASI products and the CIS charts as reference

In [ ]:
bias_r2=np.array([])
bias_asi=np.array([])

for i in range(len(cis)):
    
    date=pd.to_datetime(cis.iloc[i]['Date'])
    sic= cis.iloc[i]['SIC']
    
    bias_r2=np.append(bias_r2,(r2[pd.to_datetime(r2['Date (YYYY-MM-DD)'])==date]['Mean Ice Concentration (%)']-sic).values)
    bias_asi=np.append(bias_asi,(asi[pd.to_datetime(asi['Date (YYYY-MM-DD)'])==date]['Mean Ice Concentration (%)']-sic).values)

print('RSAT-2 bias (%): ' + str(np.nanmean(bias_r2)))
print('ASI bias (%): ' + str(np.nanmean(bias_asi)))
print('RSAT-2 RMSE (%): ' + str(np.sqrt(np.nanmean(bias_r2**2))))
print('ASI RMSE (%): ' + str(np.sqrt(np.nanmean(bias_asi**2))))

RSAT-2 bias (%): -0.4021337816085189<br>
ASI bias (%): -4.039268046759609<br>
RSAT-2 RMSE (%): 8.412943190250207<br>
ASI RMSE (%): 10.815901848767634

## Figure: Monthly bias distribution for the ASI SIC and RSAT-2 products

In [ ]:
fig1, ax1=plt.subplots(figsize=(15,10))

data1=[]
data2=[]

for i in range(1,13):

    data1.append(bias_r2[(np.isnan(bias_r2)==0) & (cis['Date'].iloc[1:].dt.month==i)])
    data2.append(bias_asi[(np.isnan(bias_asi)==0) & (cis['Date'].dt.month==i)])
    
bp1 = ax1.boxplot(data1, whis = [5, 95], sym='',patch_artist = True, 
                     widths = 0.3, positions = np.arange(0.8, 12, 1))
for box in bp1['boxes']:
    box.set(facecolor = 'darkred')
bp2 = ax1.boxplot(data2, whis = [5, 95], sym='',patch_artist = True,
                     widths = 0.3, positions = np.arange(1.2, 13, 1))
for box in bp2['boxes']:
    box.set(facecolor = 'teal')
ax1.plot([0,13],[0,0],'-k')
ax1.set_xlim(0,13)
ax1.set_ylim(-50,50)
ax1.set_xticks(np.arange(1, 13,1))
ax1.set_xticklabels(np.arange(1,13).tolist(),fontsize=22,fontweight='bold')
ax1.set_yticklabels(np.arange(-60,50,20).tolist(),fontsize=22,fontweight='bold')
ax1.set_xlabel('Month',fontsize=26,fontweight='bold')
ax1.set_ylabel('Bias (%)',fontsize=26,fontweight='bold')
ax1.legend([bp1["boxes"][0], bp2["boxes"][0]], ['RSAT-2', 'ASI'], loc='upper right', fontsize=26)




<center><img src="../../Data/figures/Figure9.png" height="500px"></center>

<center>Figure 9. Box plot of the weekly SIC biases from Figure 8 for each month for the entire time series. The CIS charts were taken as reference for the bias calculation.</center>